In [ ]:
import geopandas
import shapely
import numpy
import matplotlib.pyplot
import scipy.interpolate

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Import a single polyline and explore creating a spline through it

In [ ]:
channel = geopandas.read_file(r"C:\Users\pearsonra\Documents\data\river_bathemetry\waikanae\caches\100000000\rec_main_channel.geojson")

In [ ]:
resolution = 1

# Polyline sampling functions

In [ ]:
def get_corner_points(channel) -> numpy.ndarray:
    x = []; y = []
    for line_string in channel.geometry:
        xy = line_string.xy
        x.extend(xy[0][::-1]); y.extend(xy[1][::-1])

    xy = numpy.array([x, y])
    xy_unique, indices = numpy.unique(xy, axis=1, return_index=True)
    indices.sort(); xy = xy[:, indices]
    return xy

In [ ]:
def get_spaced_points_with_corners(channel, spacing) -> numpy.ndarray:
    xy_spaced = []
    for line_string in channel.geometry:
        
        xy_segment = line_string.xy
        x = xy_segment[0]; y = xy_segment[1]
        for i in numpy.arange(len(x) - 1, 0, -1):
            line_segment = shapely.geometry.LineString([[x[i], y[i]], [x[i - 1], y[i - 1]]])

            number_segment_samples = max(numpy.round(line_segment.length / spacing), 2)
            segment_resolution = line_segment.length / (number_segment_samples - 1)

            xy_spaced.extend([line_segment.interpolate(i * segment_resolution) for i in numpy.arange(0, number_segment_samples)])
        
    xy=numpy.array(shapely.geometry.LineString(xy_spaced).xy)
    xy_unique, indices = numpy.unique(xy, axis=1, return_index=True)
    indices.sort()
    xy = xy[:, indices]

    return xy

In [ ]:
def get_spaced_points(channel, spacing) -> numpy.ndarray:

    xy_corner_points = get_corner_points(channel)
    xy_spaced = []
    line_string = shapely.geometry.LineString(xy_corner_points.T)

    number_segment_samples = max(numpy.round(line_string.length / spacing), 2)
    segment_resolution = line_string.length / (number_segment_samples - 1)

    xy_spaced.extend([line_string.interpolate(i * segment_resolution) for i in numpy.arange(number_segment_samples - 1, -1, -1)])
        
    xy=numpy.array(shapely.geometry.LineString(xy_spaced).xy)
    xy_unique, indices = numpy.unique(xy, axis=1, return_index=True)
    indices.sort()
    xy = xy[:, indices]

    return xy

# Parametric spline fit - interpolation
Note _smoothing_multipiler_ allows for some smoothing!

In [ ]:
def fit_spline_to_points(xy, res, smoothing_multiplier) -> numpy.ndarray:
    
    smoothing_factor = smoothing_multiplier * len(xy[0])
    
    tck_tuple, u_input = scipy.interpolate.splprep(xy, s=smoothing_factor)

    # Sample every roughly res along the spine
    line_length = shapely.geometry.LineString(xy.T).length
    sample_step_u = 1 / round(line_length / res)
    u_sampled = numpy.arange(0, 1 + sample_step_u, sample_step_u)
    xy_sampled = scipy.interpolate.splev(u_sampled, tck_tuple)
    xy_sampled = numpy.array(xy_sampled)
    
    return xy_sampled

## Spline fit from corner points only
Does not follow line very well in between

In [ ]:
xy_c1

In [ ]:
xy_s1

In [ ]:
smoothing_multiplier = 50
xy_c1 = get_corner_points(channel)
xy_s1 = fit_spline_to_points(xy_c1, resolution, smoothing_multiplier)

f, ax = matplotlib.pyplot.subplots(figsize=(40,20));
channel.plot(ax=ax, linestyle='--', markersize=1, label='original', color='b', zorder=0); 
matplotlib.pyplot.plot(xy_c1[0], xy_c1[1], 'ro', markersize=5, label='sample points', zorder=2);
matplotlib.pyplot.plot(xy_s1[0], xy_s1[1], 'k-', markersize=1, label='spline', zorder=1);
matplotlib.pyplot.legend(); ax.set(title=f"Centreline from corner points")

## Spline fit from space points
Space but loses corner points
Still interpolation - Goes through each point

In [ ]:
spacing = 50; smoothing_multiplier = 50
xy_c2 = get_spaced_points(channel, spacing)
xy_s2 = fit_spline_to_points(xy_c2, resolution, smoothing_multiplier)

f, ax = matplotlib.pyplot.subplots(figsize=(40,20));
channel.plot(ax=ax, linestyle='--', markersize=1, label='original', color='b', zorder=0); 
matplotlib.pyplot.plot(xy_c2[0], xy_c2[1], 'ro', markersize=5, label='sample points', zorder=2);
matplotlib.pyplot.plot(xy_s2[0], xy_s2[1], 'k-', markersize=1, label='spline', zorder=1);
matplotlib.pyplot.legend(); ax.set(title=f"Centreline from points spaced by {spacing}")

## Spline fit from space points
Spaced but keeps corner points

In [ ]:
spacing = 50; smoothing_multiplier = 50
xy_c3 = get_spaced_points_with_corners(channel, spacing)
print(f"Number of points to fit: {len(xy_c3[0])}")
xy_s3 = fit_spline_to_points(xy_c3, resolution, smoothing_multiplier)

f, ax = matplotlib.pyplot.subplots(figsize=(40,20));
channel.plot(ax=ax, linestyle='--', markersize=1, label='original', color='b', zorder=0); 
matplotlib.pyplot.plot(xy_c3[0], xy_c3[1], 'ro', markersize=5, label='sample points', zorder=2);
matplotlib.pyplot.plot(xy_s3[0], xy_s3[1], 'k-', markersize=1, label='spline', zorder=1);
matplotlib.pyplot.legend(); ax.set(title=f"Centreline from points spaced by {spacing} with corner points and a smoothing multiplier of {smoothing_multiplier}")

In [ ]:
spacing = 50; smoothing_multiplier = 500
xy_c3 = get_spaced_points_with_corners(channel, spacing)
print(f"Number of points to fit: {len(xy_c3[0])}")
xy_s3 = fit_spline_to_points(xy_c3, resolution, smoothing_multiplier)

f, ax = matplotlib.pyplot.subplots(figsize=(40,20));
channel.plot(ax=ax, linestyle='--', markersize=1, label='original', color='b', zorder=0); 
matplotlib.pyplot.plot(xy_c3[0], xy_c3[1], 'ro', markersize=5, label='sample points', zorder=2);
matplotlib.pyplot.plot(xy_s3[0], xy_s3[1], 'k-', markersize=1, label='spline', zorder=1);
matplotlib.pyplot.legend(); ax.set(title=f"Centreline from points spaced by {spacing} with corner points and a smoothing multiplier of {smoothing_multiplier}")

# Parametric spline fit - control points
Based on: http://vadym-pasko.com/blog/2015/03/06/spline-approx-scipy.html

In [ ]:
def fit_spline_to_points_from_knots(xy, res, k=3) -> numpy.ndarray:
    
    knotspace = range(len(xy[0]))
    knots = scipy.interpolate.InterpolatedUnivariateSpline(knotspace, knotspace, k=k).get_knots()
    knots_full = numpy.concatenate(([knots[0]] * k, knots, [knots[-1]] * k))
    
    tckX = knots_full, xy[0], k
    tckY = knots_full, xy[1], k

    splineX = scipy.interpolate.UnivariateSpline._from_tck(tckX)
    splineY = scipy.interpolate.UnivariateSpline._from_tck(tckY)

    # get number of points to sample spline at
    line_length = shapely.geometry.LineString(xy.T).length
    number_of_samples = round(line_length / res)

    u_sampled = numpy.linspace(0, len(xy[0]) - 1, number_of_samples)
    x_sampled = splineX(u_sampled)
    y_sampled = splineY(u_sampled)
    
    
    
    return numpy.array([x_sampled, y_sampled])

## Spline fit from  corner points only

In [ ]:
xy_c4 = get_corner_points(channel)
xy_s4 = fit_spline_to_points_from_knots(xy_c4, resolution)

f, ax = matplotlib.pyplot.subplots(figsize=(40,20));
channel.plot(ax=ax, linestyle='--', markersize=1, label='original', color='b', zorder=0); 
matplotlib.pyplot.plot(xy_c4[0], xy_c4[1], 'ro', markersize=5, label='sample points', zorder=2);
matplotlib.pyplot.plot(xy_s4[0], xy_s4[1], 'k-', markersize=1, label='spline', zorder=1);
matplotlib.pyplot.legend(); ax.set(title=f"Centreline from corner points")

## Spline fit from spaced points

In [ ]:
spacing = 50
xy_c5 = get_spaced_points(channel, spacing)
xy_s5 = fit_spline_to_points_from_knots(xy_c5, resolution)

f, ax = matplotlib.pyplot.subplots(figsize=(40,20));
channel.plot(ax=ax, linestyle='--', markersize=1, label='original', color='b', zorder=0); 
matplotlib.pyplot.plot(xy_c5[0], xy_c5[1], 'ro', markersize=5, label='sample points', zorder=2);
matplotlib.pyplot.plot(xy_s5[0], xy_s5[1], 'k-', markersize=1, label='spline', zorder=1);
matplotlib.pyplot.legend(); ax.set(title=f"Centreline from corner points")

## Spline fit from spaced points with corners

In [ ]:
spacing = 50
xy_c6 = get_spaced_points_with_corners(channel, spacing)
xy_s6 = fit_spline_to_points_from_knots(xy_c6, resolution)

f, ax = matplotlib.pyplot.subplots(figsize=(40,20));
channel.plot(ax=ax, linestyle='--', markersize=1, label='original', color='b', zorder=0); 
matplotlib.pyplot.plot(xy_c6[0], xy_c6[1], 'ro', markersize=5, label='sample points', zorder=2);
matplotlib.pyplot.plot(xy_s6[0], xy_s6[1], 'k-', markersize=1, label='spline', zorder=1);
matplotlib.pyplot.legend(); ax.set(title=f"Centreline from corner points")